# IFT6285 (TALN) 

## Devoir 3: Correction des mots

Réalisé par:

Mouna Dhaouadi 

Kacem Khaled

Soumis à:

Prof. Philippe Langlais

Frédéric Piedboeuf


**Automne 2021**

In [1]:
!pip install editdistance

In [2]:
!pip install soundex

In [3]:
!pip install textdistance

In [2]:
#import jaro
import heapq as hq
import editdistance
#import nltk
import soundex
import textdistance
from soundex import Soundex
import pandas as pd
import argparse
import os.path as osp
from os.path import dirname, abspath
import logging

logging.basicConfig(format='%(message)s', level=logging.DEBUG)

In [40]:
vocab = 'voc-1bwc.txt'
wrong_words = 'devoir3-train-short.txt'
reference = 'devoir3-train.txt'

CONFIG_PATH = abspath('./')
SRC_ROOT = dirname(CONFIG_PATH)
VOCAB_ROOT = osp.join(SRC_ROOT, vocab)
TRAIN_SHORT_ROOT = osp.join(SRC_ROOT, wrong_words)
TRAIN_ROOT = osp.join(SRC_ROOT, reference)

## Question 1 : Correction

In [6]:
def get_frequency_table(file_name):
    frequency_table = {}
    # create a dict from lexique
    with open(file_name ,'r',encoding="utf8") as f:
        lines = f.readlines()
        for line in lines:
            s = line.rstrip('\n').lstrip()
            frequency = int(s.split(' ')[0])
            word = s.split(' ')[1]
            frequency_table[word] = frequency
    f.close()
    return frequency_table

In [29]:
def find_best_candidates(wrong_word, vocab_scores, nb_best_scores, largest, n=5, order_by= 'unigram'): 
    scores = list(set(a[0] for a in vocab_scores.values())) 
    if largest:
     # Find three biggest scores (sorted) --> three best possible corrections
        best_scores= hq.nlargest(nb_best_scores, scores)
    else:
        best_scores=  hq.nsmallest(nb_best_scores, scores)
            
    best_candidates = {v:vocab_scores[v] for v in vocab_scores if vocab_scores[v][0] in best_scores}
    
    if order_by == 'distance':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][0]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]
    elif order_by == 'unigram':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][1]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]
    elif order_by == 'comb_d_u':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][0],best_candidates[k][1]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]
    elif order_by == 'comb_u_d':
        best_candidates = sorted(best_candidates, key=lambda k: (best_candidates[k][1],best_candidates[k][0]), reverse=largest)
        #print(best_candidates[:30])
        corrections = [c for c in best_candidates][:n]

    affichage(wrong_word, corrections)
    
    return corrections, best_candidates

In [30]:
def affichage(wrong_word, corrections):
    if len(corrections) == 0 : 
        result =  wrong_word + '\t' + wrong_word
    else:
        result =  wrong_word + '\t' + '\t'.join(corrections)
    print(result)

In [31]:
def process(dispatcher,distance_name,wrong_word,frequency_table,vocab,ordre):
    
    if distance_name=="Soundex":
        instance = Soundex()
        # We want Englidh corrections --> remove non english corrections (-1)
        vocab = [ v for v in vocab if instance.compare(v, wrong_word) >= 0 ]
    distance_scores = { v: [dispatcher[distance_name][0](v, wrong_word), frequency_table[v]] for v in vocab }
    logging.info(f'Corrections using {distance_name} distance :')
    find_best_candidates(wrong_word, distance_scores, 3, dispatcher[distance_name][1],5, order_by= ordre) # 'distance' , 'unigram' or 'comb_d_u' or 'comb_u_d'


In [49]:
def corrige(wrong_words, lexique_file, distances=['Jaro_Winkler','Levenshtein'], order_by='unigram'):

    frequency_table = get_frequency_table(lexique_file)
    vocab = list(frequency_table.keys())
    instance = Soundex()

    # https://pypi.org/project/textdistance/
    dispatcher = {
        'Jaro_Winkler' : [textdistance.jaro_winkler.distance,False],
        'Levenshtein' : [textdistance.levenshtein.distance,False],
        'Jaccard': [textdistance.jaccard.distance,False],
        'Cosine': [textdistance.cosine.distance,False],
        'Hamming': [textdistance.hamming.distance,False],
        'LCSS': [textdistance.lcsstr.distance,False], # textdistance.lcsstr.similarity,True
        'Damerau_Levenshtein': [textdistance.damerau_levenshtein.distance,False],
        'Needleman_Wunsch': [textdistance.needleman_wunsch.distance,False],
        'Soundex': [instance.compare,False]
                 }
    

    # lit une liste de mots a corriger, un par ligne
    with open(wrong_words, 'r',encoding="utf8") as f:
        for line in f:

            wrong_word = line.rstrip().split()[0]
            logging.info('********************************* ' + wrong_word)
            # calculate the distance for all the vocabulary
            # for each word in vocabulary, create a dictionary of candidates with 'nb_best_scores': {cand1:score1,cand2:score2,...}
            # get the 'n' best candidates and order them according to a criteria: 'distance' , 'unigram' or 'comb_d_u' or 'comb_u_d'
            
            for d in distances:
                process(dispatcher,d,wrong_word,frequency_table,vocab,order_by)

    f.close()

In [50]:
# distances = ['Jaro_Winkler', 'Levenshtein', 'Jaccard', 'Cosine', 'Hamming', 'LCSS', 'Damerau_Levenshtein', 'Needleman_Wunsch', 'Soundex']
distances = ['Jaro_Winkler', 'Levenshtein', 'Jaccard', 'Cosine', 'Hamming', 'LCSS', 'Damerau_Levenshtein', 'Needleman_Wunsch', 'Soundex']
corrige(wrong_words, lexique,distances, order_by='distance')


********************************* reccodmission


Corrections using Jaro_Winkler distance :
reccodmission	recommissioned	comission	re-admission
Corrections using Levenshtein distance :
reccodmission	re-admission	decommission	readmission	re-admissions	recission
Corrections using Jaccard distance :
reccodmission	microseconds	microsecond	recommissioned	semiconductors
Corrections using Cosine distance :
reccodmission	microseconds	microsecond	recommissioned	semiconductors
Corrections using Hamming distance :
reccodmission	zero-emission	co-commissioned	enviromission	high-emission	multi-mission
Corrections using LCSS distance :
reccodmission	re-admissions	re-admission	readmissions	readmission	admissions
Corrections using Damerau_Levenshtein distance :
reccodmission	re-admission	decommission	readmission	re-admissions	recission
Corrections using Needleman_Wunsch distance :
reccodmission	re-admission	decommission	re-admissions	recommissioned	resubmission


********************************* unpair


Corrections using Soundex distance :
reccodmission	cost-to-income	tostenson	boston.com	hosting.com	distension
Corrections using Jaro_Winkler distance :
unpair	unpaid	unfair	unai
Corrections using Levenshtein distance :
unpair	unpaid	unfair	usair	in-air	umair
Corrections using Jaccard distance :
unpair	purina	nirupam	pianura	manipur	puritan
Corrections using Cosine distance :
unpair	purina	nirupam	pianura	manipur	puritan
Corrections using Hamming distance :
unpair	unpaid	unfair	in-air	unsaid	unpack
Corrections using LCSS distance :
unpair	unpaid	pairc	unpack	impair	paired
Corrections using Damerau_Levenshtein distance :
unpair	unpaid	unfair	usair	in-air	umair
Corrections using Needleman_Wunsch distance :
unpair	unpaid	unfair	in-air	funfair	unsaid


********************************* oconomic


...

Corrections using Soundex distance :
produse	artuso	barrettes	birdcages	bredius	brodies
Corrections using Jaro_Winkler distance :
tecnologycal	technological	technology	technologically
Corrections using Levenshtein distance :
tecnologycal	technological	teleological	ecological	ethnological	selenological
Corrections using Jaccard distance :
tecnologycal	technologically	gynecological	technological	clean-technology	colangelo
Corrections using Cosine distance :
tecnologycal	technologically	gynecological	technological	clean-technology
Corrections using Hamming distance :
tecnologycal	teleological	ethnological	phonological	sociological	neurological
Corrections using LCSS distance :
tecnologycal	vaccinology	oenology	technology-	synology	enology
Corrections using Damerau_Levenshtein distance :
tecnologycal	technological	ethnological	teleological	ecological	selenological
Corrections using Needleman_Wunsch distance :
tecnologycal	technological	teleological	ethnological	selenological	phonological


********************************* upto


Corrections using Soundex distance :
tecnologycal	cosmological	seismological	technologically	technological	aaaa
Corrections using Jaro_Winkler distance :
upto	upto	upton	uspto
Corrections using Levenshtein distance :
upto	upto	updo	pto	uspto	unto
Corrections using Jaccard distance :
upto	oput	upto	pout	stoup	toups
Corrections using Cosine distance :
upto	oput	upto	pout	stoup	toups
Corrections using Hamming distance :
upto	upto	updo	unto	upton	anto
Corrections using LCSS distance :
upto	upto	ptoi	pto	supt	upton
Corrections using Damerau_Levenshtein distance :
upto	upto	updo	pto	uspto	unto
Corrections using Needleman_Wunsch distance :
upto	upto	updo	uspto	unto	upton


********************************* poisioning


Corrections using Soundex distance :
upto	upto	abotu	abutu	biffed	cavitt
Corrections using Jaro_Winkler distance :
poisioning	poisoning	provisioning	positioning
Corrections using Levenshtein distance :
poisioning	poisoning	poisonings	provisioning	positioning	co-signing
Corrections using Jaccard distance :
poisioning	positioning	poisoning	provisioning
Corrections using Cosine distance :
poisioning	positioning	poisoning	provisioning
Corrections using Hamming distance :
poisioning	co-signing	moistening	consigning	postponing	roistering
Corrections using LCSS distance :
poisioning	decisioning	envisioning	optioning	sectioning	suctioning
Corrections using Damerau_Levenshtein distance :
poisioning	poisoning	poisonings	provisioning	positioning	co-signing
Corrections using Needleman_Wunsch distance :
poisioning	provisioning	positioning	poisoning	co-signing	moistening
Corrections using Soundex distance :
poisioning	agenc	ausenco	bushisms	ccms	cieszynski


In [21]:
textdistance.cosine.distance('test', 'text') 

0.25

In [ ]:
# Votre programme prendra comme seul argument obligatoire le nom d'un lexique de mots 
# Vous pouvez doter votre programme d'options permettant de contr^oler le comportement de votre correcteur. 

# TODO a seperate python script that we can run with the provided command in TP
distances = ['Jaro_Winkler', 'Levenshtein', 'Jaccard', 'Cosine', 'Hamming', 'LCSS', 'Damerau_Levenshtein', 'Needleman_Wunsch', 'Soundex']
order_by = ['distance','unigram','comb_d_u','comb_u_d']

parser = argparse.ArgumentParser(description='Words correction')
parser.add_argument('-v','--vocab', default=VOCAB_ROOT,
                    help='Path to vocab file: (default: ./voc-1bwc.txt)')


parser.add_argument('-w','--wrong_words', default=TRAIN_SHORT_ROOT,
                    help='Path to wrong words file: (default: ./devoir3-train-short.txt)')

parser.add_argument('-d', '--distance', nargs='+', default=['Jaro_Winkler'],
                    choices=distances,
                    help='Distances: ' +
                        ' | '.join(distances) +
                        ' (default: Jaro_Winkler)')

parser.add_argument('-o', '--order', default='unigram',
                    choices=order_by,
                    help='Order by: ' +
                        ' | '.join(order_by) +
                        ' (default: unigram)')


def main():
    args = parser.parse_args()
    vocab = osp.join(SRC_ROOT, args.vocab)
    wrong_words = osp.join(SRC_ROOT, args.wrong_words)
    corrige(wrong_words, vocab, args.distance, args.order)

if __name__ == '__main__':
    main()

# python corrige.py -v voc-1bwc.txt -w devoir3-train-short.txt -d Jaro_Winkler -o unigram > devoir3-sortie.txt

In [44]:
!python corrige.py -v voc-1bwc.txt -w devoir3-train-short.txt -d Jaro_Winkler -o unigram > devoir3-sortie.txt

********************************* reccodmission
Corrections using Jaro_Winkler distance :
********************************* unpair
Corrections using Jaro_Winkler distance :
********************************* oconomic
Corrections using Jaro_Winkler distance :
********************************* accidnts
Corrections using Jaro_Winkler distance :
********************************* succes
Corrections using Jaro_Winkler distance :
********************************* enthousiastic
Corrections using Jaro_Winkler distance :
********************************* produse
Corrections using Jaro_Winkler distance :
********************************* tecnologycal
Corrections using Jaro_Winkler distance :
********************************* upto
Corrections using Jaro_Winkler distance :
********************************* poisioning
Corrections using Jaro_Winkler distance :


## Evaluation

In [ ]:
reference = 'devoir3-train.txt'

In [ ]:
def eval(correction_line, reference):
    # look for wrong word in the reference file
    
    # take the correct correctionss
    
    # compute mesure de qualite ? 
    pass
    

In [1]:
{i:i for i in range(10)}

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [13]:
instance = Soundex()
instance.compare('cost-to-income','reccodmission')

1

In [35]:
distance_name ='Soundex'

In [36]:
nb_top_scores= 3 if distance_name != 'Soundex' else 1

In [37]:
nb_top_scores

1